In [1]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd
import numpy as np

In [2]:
def getResults():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, r.featureactionid, rv.valueid, 
                                       rv.datavalue, rv.valuedatetime, sf.samplingfeaturecode,
                                       ds.datasetcode
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid 
                       INNER JOIN dataset_results dr ON r.resultid=dr.resultid
                       INNER JOIN datasets ds ON dr.datasetid=ds.datasetid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE ds.datasetcode LIKE 'T1Data_%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'NC%' ORDER BY featureactionid
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'featureactionid', 'valueid', 
                                     'datavalue', 'valuedatetime', 'samplingfeaturecode', 'datasetcode']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'featureactionid': row[2],
                            'valueid': row[3], 'datavalue': row[4], 'valuedatetime': row[5], 
                            'samplingfeaturecode': row[6], 'datasetcode': row[7]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
dfdb = getResults()
dfdb.head()

,resultid,resultdatetime,featureactionid,valueid,datavalue,valuedatetime,samplingfeaturecode,datasetcode
0,31041,2019-07-10 12:00:00-04:00,1024,21013,8.635,2019-07-10 12:00:00-04:00,NC0136025-1_Specimen,T1Data_Knappe_NC0136025-1_Specimen
1,34200,2019-06-25 12:00:00-04:00,1454,24172,2.500,2019-06-25 12:00:00-04:00,NC0464010_Specimen,T1Data_Knappe_NC0464010_Specimen
2,35030,2019-08-20 12:00:00-04:00,1508,25002,1.300,2019-08-20 12:00:00-04:00,NC0496010_Specimen,T1Data_Ferguson_NC0496010_Specimen
3,34410,2019-05-29 12:00:00-04:00,1467,24382,9.900,2019-05-29 12:00:00-04:00,NC0465010-1_Specimen,T1Data_Ferguson_NC0465010-1_Specimen
4,35311,2019-08-06 12:00:00-04:00,1549,25283,6.200,2019-08-06 12:00:00-04:00,NC6054001_Specimen,T1Data_Ferguson_NC6054001_Specimen


In [18]:
len(dfdb['resultid'])

815

In [24]:
outfile = '../resultsT2Data.csv'
dfdb.to_csv(outfile, header=True, index=False)

In [4]:
len(np.unique(np.array(dfdb['valueid'])))

18171

In [5]:
len(dfdb['valueid'])

18171

In [6]:
len(np.unique(np.array(dfdb['resultid'])))

18171

In [7]:
len(dfdb['resultid'])

18171

In [8]:
def getActions():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'NC%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [9]:
dfdb = getActions()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid
0,20619,2019-08-12 12:00:00-04:00,10591,1.0,2019-08-12 12:00:00-04:00,403,2019-08-12 00:00:00-04:00,2019-08-12 23:59:59-04:00,10005
1,15037,2019-05-07 12:00:00-04:00,5009,1.0,2019-05-07 12:00:00-04:00,284,2019-05-07 00:00:00-04:00,2019-05-07 23:59:59-04:00,10003
2,12421,2019-07-17 12:00:00-04:00,2393,2.0,2019-07-17 12:00:00-04:00,228,2019-07-17 00:00:00-04:00,2019-07-17 23:59:59-04:00,10003
3,14175,2019-08-20 12:00:00-04:00,4147,1.0,2019-08-20 12:00:00-04:00,265,2019-08-20 00:00:00-04:00,2019-08-20 23:59:59-04:00,10003
4,13363,2019-04-10 12:00:00-04:00,3335,1.0,2019-04-10 12:00:00-04:00,248,2019-04-10 00:00:00-04:00,2019-04-10 23:59:59-04:00,10003


In [10]:
len(np.unique(np.array(dfdb['actionid'])))

380

In [11]:
len(dfdb['actionid'])

18171

In [12]:
len(np.unique(np.array(dfdb['end_datetime'])))

44

In [13]:
def getFeatures():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid,
                                        sf.samplingfeatureid, ST_X(ST_AsText(sf.featuregeometry))
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'NC%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid',
                                      'samplingfeatureid', 'lon']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8],
                            'samplingfeatureid': row[9], 'lon': row[10]},
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [14]:
dfdb = getFeatures()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid,samplingfeatureid,lon
0,10919,2019-07-02 12:00:00-04:00,891,1.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
1,10920,2019-07-02 12:00:00-04:00,892,5.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
2,10921,2019-07-02 12:00:00-04:00,893,10.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
3,10922,2019-07-02 12:00:00-04:00,894,2.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
4,10923,2019-07-02 12:00:00-04:00,895,2.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111


In [15]:
np.array(dfdb['lon'])

array([-79.47892666, -78.02686657, -76.35896649, ..., -77.30003197,
       -82.22916199, -78.74967816])

In [16]:
len(np.unique(np.array(dfdb['samplingfeatureid'])))

380